![title](szeged.png)

Szeged (/ˈsɛɡɛd/ SEG-ed, Hungarian: is the third largest city of Hungary, the largest city and regional centre of the Southern Great Plain and the county seat of Csongrád-Csanád county-

Content

The CSV file includes a hourly/daily summary for Szeged, Hungary area, between 2006 and 2016.

Data available in the hourly response:

    1.Time
    2.Summary
    3.PrecipType
    4.Temperature
    5.ApparentTemperature
    6.Humidity
    7.WindSpeed
    8.WindBearing
    9.Visibility
    10.LoudCover
    11.Pressure


# Please do note that this notebook is not yet run, because the size too big for github to read if it is saved after a run.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime

# Set the plots to display in the Jupyter notebook
%matplotlib inline

# Plotting colors per client request
plt.style.use('ggplot')

# Increase default figure and font sizes for easier viewing.
plt.rcParams['figure.figsize'] = (12, 5)
plt.rcParams['font.size'] = 10

Database link: https://www.kaggle.com/budincsevity/szeged-weather

In [ ]:
# Reading the data file
weather = pd.read_csv('weather_szeged_2006-2016.csv')

In [ ]:
# Show some of value inside the DF
weather.head(3)

In [ ]:
# Stating different types of data for each columns in the DF
weather.dtypes

In [ ]:
# Show the number of columns & rows in the DF
weather.shape

In [ ]:
# Calculate the rough total number of rows in the dataframe
def  row_amt_mult(*args):
    product = 1
    
    for num in args:
        product = product * num
    return product

row_amt_mult(24,365,11)

Diffence between 'weather.shape' & 'row_amt_mult' row count output is because there are some repeated row index which will be drop in later step.

In [ ]:
# A function to get the information for specific medium
def df_info():
    while True:
        try:
            input_medium = int(input( " Choose 1: Shape\n Choose 2: Columns\n Choose 3: Data types\n Choose 4: Index\n Choose 5: Describe\n Choose 6: Head\n Choose 0: Exit\n "))
            
            if input_medium == 1:
                print ("This are the rows and columns of the DataFrame ",weather.shape)
            if input_medium == 2:
                print ("This are the list of columns for the DataFrame \n \n",weather.columns)
            if input_medium == 3:
                print ("This are the data types for the columns in DataFrame \n\n",weather.dtypes)
            if input_medium == 4:
                print( "The DataFrame is currently using this as index\n\n",weather.index) 
            if input_medium == 5:
                print("This are the summary of the DataFrame\n\n",weather.describe())
            if input_medium == 6:
                print("This are some of  the data from the DataFrame\n\n",weather.head())
            if input_medium == 0:
                break
    
        except Exception as ex:
            print("Ooops! There is an error: {} with exception: {}". format(type(ex).__name__,ex.args))
            continue 

df_info()

In [ ]:
# Creat a class for filtering purposes for a better valuesvizualization
class Filter:
    def __init__(self, make):
        self.make = make
        print(self.make)
 
# Filter by column
    def filter_col(self, weather_df_test):
        self.test = weather
        self.weather_df_test = weather_df_test
        return self.test[self.weather_df_test]

# Filter by requirement
    def filter_req(self, weather_df_test, the_object):
        list_observe = []
        observe = ''
        while observe != 'all':
            observe = input( " Enter column name : ")
            if observe !='all':
                list_observe.append(observe)
                return self.test[self.test[weather_df_test] == the_object][[observe]]
            if observe == 'all':
                return self.test[self.test[weather_df_test] == the_object]       

In [ ]:
# Create object out of the Filter class
df_one = Filter('create_df')

In [ ]:
# Utilize the filter column method
df_one.filter_col(['Temperature (C)'])

In [ ]:
# Utilize the specific requirement filter method
# Please enter a column name in the input space
df_one.filter_req('Precip Type', 'snow')

What is UTC: Coordinated Universal Time or UTC is the primary time standard by which the world regulates clocks and time.      -Wikipedia-

In [ ]:
weather.head(3)

In [ ]:
# Deleting the UTC +0200 from the 'Formatted Date' in order to set this column as the index
weather['Formatted Date'] = weather['Formatted Date'].apply(lambda x:x[:-6])

In [ ]:
# Convert 'Formatted Date' from object to DateTime
weather['Formatted Date'] = pd.to_datetime(weather['Formatted Date'])

In [ ]:
# Set the 'Formatted Date as index
weather.set_index('Formatted Date', inplace=True)

In [ ]:
weather.head(3)

ANALYSIS: Study between Temperature (C) & Apparent Temperature (C)

Apparent temperature is the temperature equivalent perceived by humans, caused by the combined effects of air temperature, relative humidity and wind speed. Apparent temperature was invented by Robert Steadman who published a paper about it in 1984. -Wikipedia-

In [ ]:
# Create sub DF for later temperature analysis
weather_temp = df_one.filter_col(['Temperature (C)','Apparent Temperature (C)'])
#weather_temp = weather[['Temperature (C)', 'Apparent Temperature (C)']]

print(weather_temp.head())
print(weather_temp.shape)

In [ ]:
# Create a variable for daily average temperatures
avg_daily_temp = weather_temp.resample('24H').mean()


print('Average Daily Temperature')
print(avg_daily_temp)
print('__________________________________________________________')
print(avg_daily_temp.describe())

avg_daily_temp.plot(kind='box')

In [ ]:
plt.figure(figsize=(28,10))

# Create variable for daily average temp for 1 year
oneyear_avg_daily_temp = avg_daily_temp.groupby(avg_daily_temp.index.day_of_year).mean()

# Plotting for 1 year daily average temperature of Temperature & Apparent Temperature
plt.plot(oneyear_avg_daily_temp.index, oneyear_avg_daily_temp['Temperature (C)'], label=('Average Daily Temperature'))
plt.plot(oneyear_avg_daily_temp.index, oneyear_avg_daily_temp['Apparent Temperature (C)'], label=('Average Daily Apparent Temperature'))

plt.title('1 Year Average Temperature (C) & Apparent Temperature (C) for Szeged, 2006', fontsize=20)
plt.xticks(np.linspace(0,366,13)[:-1], ('Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun','Jul','Aug','Sep','Oct', 'Nov', 'Dec'), fontsize=15)
plt.yticks(fontsize=15)
plt.legend()

print('\nFrom the below line chart we see that there are obvious deviations between Temperature & Apparent Temperature from a period of November to April')

# Create variable for monthly average temp
# Create variable for monthly average temp for 11 years
avg_monthly_temp = weather_temp.resample('1M').mean()
elevenyears_avg_monthly_temp = avg_monthly_temp.groupby(avg_monthly_temp.index).mean()
plt.figure(figsize=(28,15))

# Plotting for 11 years monthly average temperature of Temperature & Apparent Temperature
plt.plot(elevenyears_avg_monthly_temp.index, elevenyears_avg_monthly_temp['Temperature (C)'], label=('Average Montly Temperature'))
plt.plot(elevenyears_avg_monthly_temp.index, elevenyears_avg_monthly_temp['Apparent Temperature (C)'], label=('Average Monthly Apparent Temperature'))

plt.title('11 Years Temperature (C) & Apparent Temperature (C) for Szeged, 2006-2016', fontsize=20)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.legend()

From the above line charts, it shows that deviations happened on early & at end of the year.
So why does this deviation happens at that specific time?. Does this correlate with what Robert Steadman said? air temperature, relative humidity and wind speed are what makes the Apparent Temperature that human body perceives?.

In [ ]:
# Create sub DF fro humidity
#weather_humidity = weather[['Humidity']]
weather_humidity = df_one.filter_col(['Humidity'])

In [ ]:
weather_humidity

In [ ]:
# Create a veriable for daily average humidty
avg_daily_humidity = weather_humidity.resample('24H').mean()

print('Average Daily Humidity')
print(avg_daily_humidity)
print('__________________________________________________________')
print(avg_daily_humidity.describe())

avg_daily_humidity.plot(kind='box')

In [ ]:
plt.figure(figsize=(28,10))

# Create variable for daily average humidity for 1 year
oneyear_avg_daily_humidity = avg_daily_humidity.groupby(avg_daily_humidity.index.day_of_year).mean()

# Plotting for 1 year daily average humidity
plt.plot(oneyear_avg_daily_humidity.index, oneyear_avg_daily_humidity, label=('Average Daily Humidity'))

plt.title('1 Year Humidity for Szeged, 2006', fontsize=20)
plt.xticks(np.linspace(0,366,13)[:-1], ('Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun','Jul','Aug','Sep','Oct', 'Nov', 'Dec'), fontsize=15)
#plt.yticks(range(-10, 120, 10), fontsize=15)
plt.legend()

#print('\nFrom the below line chart we see that there are obvious deviations between Temperature & Apparent Temperature from a period of November to April')


plt.figure(figsize=(28,10))

# Create variable for monthly average humidity
# Create variable for monthly average humidity for 11 years
avg_monthly_humidity = weather_humidity.resample('1M').mean()
elevenyears_avg_monthly_humidity = avg_monthly_humidity.groupby(avg_monthly_humidity.index).mean()

# Plotting for 11 years monthly average humidity
plt.plot(elevenyears_avg_monthly_humidity.index, elevenyears_avg_monthly_humidity, label=('Average Montly Humidity'))

plt.title('11 Years Humidity for Szeged, 2006-2016', fontsize=20)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.legend()



Line graph from above shows that in the early & end of the year, humidity are high.

In [ ]:
# Create sub DF fro humidity
#weather_wind= weather[['Wind Speed (km/h)']]
weather_wind = df_one.filter_col(['Wind Speed (km/h)'])

In [ ]:
weather_wind

In [ ]:
# Create a veriable for daily average wind speed
avg_daily_wind = weather_wind.resample('24H').mean()

print('Average Daily Wind Speed')
print(avg_daily_wind)
print('__________________________________________________________')
print(avg_daily_wind.describe())

avg_daily_wind.plot(kind='box')

In [ ]:
plt.figure(figsize=(28,10))

# Create variable for daily average wind speed for 1 year
oneyear_avg_daily_wind = avg_daily_wind.groupby(avg_daily_wind.index.day_of_year).mean()

# Plotting for 1 year daily average humidity
plt.plot(oneyear_avg_daily_wind.index, oneyear_avg_daily_wind, label=('Average Daily Wind Speed'))

plt.title('1 Year Wind Speed for Szeged, 2006', fontsize=20)
plt.xticks(np.linspace(0,366,13)[:-1], ('Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun','Jul','Aug','Sep','Oct', 'Nov', 'Dec'), fontsize=15)
#plt.yticks(range(-10, 120, 10), fontsize=15)
plt.legend()

#print('\nFrom the below line chart we see that there are obvious deviations between Temperature & Apparent Temperature from a period of November to April')


plt.figure(figsize=(28,10))

# Create variable for monthly average wind speed
# Create variable for monthly average wind speed for 11 years
avg_monthly_wind = weather_wind.resample('1M').mean()
elevenyears_avg_monthly_wind = avg_monthly_wind.groupby(avg_monthly_wind.index).mean()

# Plotting for 11 years monthly average humidity
plt.plot(elevenyears_avg_monthly_wind.index, elevenyears_avg_monthly_wind, label=('Average Montly Wind Speed'))

plt.title('11 Years Wind Speed for Szeged, 2006-2016', fontsize=20)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)
plt.legend()

From both of the wind speed line graph, we can see that the value constantly moves up & down drastically, but we can still see that in the '1 Year Wind Speed for Szeged, 2006' line graph the value from december until next year april is higher than the other months. This concludes that what Robert Steadman said in his paper from 1984 is true! From my investigation the correlation between air temperature, humidy & wind speed did affect the temperature.

In [ ]:
weather.shape

In [ ]:
weather.index.unique()

In [ ]:
# In order to create a dictionary based on weather DF index, i need to drop duplicates in the index
weather['index'] = weather.index
weather = weather.drop_duplicates('index')
weather.set_index = weather['index']
weather= weather.drop('index', axis =1)

In [ ]:
# Create dictionary out of weather DF
weather_dict_index = weather.to_dict(orient='index')

In [ ]:
weather_dict_index

In [ ]:
weather_dict_index.keys()

In [ ]:
# Calling all column values using dictionary
weather_dict_index[pd.Timestamp('2012-04-11 12:00:00')]

In [ ]:
# Calling all collumn values using 'loc'
weather.loc['2006-1-1 1:0:0' : '2006-1-1 2:0:0']

# THANK YOU